# Noisy DQN — Explorare prin Zgomot Parametric

**Noisy DQN** înlocuiește mecanismul clasic de explorare *ε-greedy* cu un mod
mult mai stabil și inteligent: **explorarea prin zgomot injectat direct în greutățile rețelei**.
În loc să aleagă uneori acțiuni random, rețeaua generează Q-values ușor diferite
datorită zgomotului din parametri.

Rezultat: agentul explorează natural, fără epsilon și fără tuning manual.

---

## 🎯 De ce Noisy Networks?

Limitările ε-greedy:
- explorare nenaturală (salturi bruște între acțiuni)
- necesită programare manuală a epsilon-ului
- performanță slabă în probleme complexe

Avantajele Noisy Nets:
- explorare **dependentă de stare**
- magnitudinea zgomotului este **învățată automat**
- agentul decide singur cât explorează sau exploatează

---

## 🧠 Noisy Linear Layer (NoisyDense)

Un layer obișnuit calculează:

$$
y = Wx + b
$$

Un layer *noisy* folosește parametri perturbați:

$$
y = (\mu_W + \sigma_W \odot \epsilon_W)\, x \;+\; 
    (\mu_b + \sigma_b \odot \epsilon_b)
$$

unde:
- **μ** — componenta deterministică  
- **σ** — mărimea zgomotului (învățabilă)  
- **ε** — zgomot Gaussian factorized  
- zgomotul se regenerează la fiecare forward pass

Acest zgomot **factorizat** este stabil și eficient — folosit în Rainbow DQN.

---

## 🔄 Mecanismul de explorare

Zgomotul din greutăți face ca Q-values să varieze la fiecare apel:

- nu mai avem *ε-random actions*
- explorarea rezultă din modificarea naturală a Q-values
- politica devine stocastică prin însăși rețeaua neuronală

Orice apel la `model(x)` include implicit explorare.

---

## 🧩 Compatibilitate cu alte îmbunătățiri

Noisy DQN se combină foarte bine cu:
- **Double DQN**
- **Dueling Networks**
- **PER**
- **Multi-Step Learning**
- **Rainbow DQN**

În laboratorul tău, versiunea implementată este **Dueling + Noisy + Double**.

---

## ⭐ Avantaje față de ε-greedy

- explorare inteligentă și adaptivă  
- nu necesită decay pentru epsilon  
- mai stabil în environment-uri cu acțiuni multiple  
- componentă esențială în **Rainbow**  
- permite un comportament explorator „continuum”, nu brusc

---

## 📌 Rezumat

**Noisy DQN = DQN + explorare învățabilă direct în greutățile rețelei.**  
Agentul devine explorator prin arhitectură, fără zgomot extern sau reguli artificiale.

Este una dintre cele mai elegante și eficiente metode moderne de explorare în RL.


In [ ]:
import gymnasium as gym
import tensorflow as tf
import numpy as np
import random
import matplotlib.pyplot as plt
from IPython.display import clear_output
from tensorflow.keras import Model, layers, optimizers


# ============================================================
# Noisy Linear Layer (Factorized Gaussian Noise)
# ============================================================

class NoisyDense(layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.units = units

    def build(self, input_shape):
        n_in = int(input_shape[-1])

        # Trainable parameters
        self.mu_w = self.add_weight(
            name="mu_w",
            shape=(n_in, self.units),
            initializer=tf.random_uniform_initializer(-1/np.sqrt(n_in), 1/np.sqrt(n_in)),
            trainable=True,
        )
        self.sigma_w = self.add_weight(
            name="sigma_w",
            shape=(n_in, self.units),
            initializer=tf.constant_initializer(0.5 / np.sqrt(n_in)),
            trainable=True,
        )

        self.mu_b = self.add_weight(
            name="mu_b",
            shape=(self.units,),
            initializer=tf.random_uniform_initializer(-1/np.sqrt(n_in), 1/np.sqrt(n_in)),
            trainable=True,
        )
        self.sigma_b = self.add_weight(
            name="sigma_b",
            shape=(self.units,),
            initializer=tf.constant_initializer(0.5 / np.sqrt(n_in)),
            trainable=True,
        )

    def call(self, x):
        # Factorized Gaussian noise
        eps_in = tf.random.normal((self.mu_w.shape[0],))
        eps_out = tf.random.normal((self.units,))

        f_in = tf.sign(eps_in) * tf.sqrt(tf.abs(eps_in))
        f_out = tf.sign(eps_out) * tf.sqrt(tf.abs(eps_out))

        # Outer product f_in ⊗ f_out → shape (n_in, units)
        w_noise = tf.tensordot(f_in, f_out, axes=0)
        b_noise = f_out

        w = self.mu_w + self.sigma_w * w_noise
        b = self.mu_b + self.sigma_b * b_noise

        return tf.matmul(x, w) + b



# ============================================================
# Replay Buffer (same structure as before)
# ============================================================

class ReplayBuffer:
    def __init__(self, size, state_dim):
        self.size = size
        self.ptr = 0
        self.full = False

        self.states = np.zeros((size, state_dim), dtype=np.float32)
        self.next_states = np.zeros((size, state_dim), dtype=np.float32)
        self.actions = np.zeros(size, dtype=np.int32)
        self.rewards = np.zeros(size, dtype=np.float32)
        self.dones = np.zeros(size, dtype=np.float32)

    def store(self, s, s2, a, r, d):
        self.states[self.ptr] = s
        self.next_states[self.ptr] = s2
        self.actions[self.ptr] = a
        self.rewards[self.ptr] = r
        self.dones[self.ptr] = d

        self.ptr = (self.ptr + 1) % self.size
        if self.ptr == 0:
            self.full = True

    def sample(self, batch_size):
        max_size = self.size if self.full else self.ptr
        idx = np.random.choice(max_size, batch_size, replace=False)
        return (
            self.states[idx],
            self.next_states[idx],
            self.actions[idx],
            self.rewards[idx],
            self.dones[idx],
        )


# ============================================================
# Noisy DQN Network
# ============================================================

class NoisyDQN(Model):
    def __init__(self, n_actions):
        super().__init__()
        self.fc1 = NoisyDense(128)
        self.fc2 = NoisyDense(128)
        self.fc3 = NoisyDense(n_actions)

    def call(self, x):
        x = tf.convert_to_tensor(x, dtype=tf.float32)
        x = tf.nn.relu(self.fc1(x))
        x = tf.nn.relu(self.fc2(x))
        return self.fc3(x)

    def act(self, state):
        state = state[np.newaxis, :]
        Q = self.call(state)[0]
        return int(tf.argmax(Q).numpy())


# ============================================================
# Hyperparameters
# ============================================================

GAMMA = 0.99
LR = 3e-4
BATCH = 32
MEM_SIZE = 50000
NUM_EPISODES = 600

# No epsilon decay — Noisy Nets handle exploration automatically


# ============================================================
# Training Loop — Noisy DQN
# ============================================================

env = gym.make("CartPole-v1")
state_dim = env.observation_space.shape[0]
n_actions = env.action_space.n

buffer = ReplayBuffer(MEM_SIZE, state_dim)
online = NoisyDQN(n_actions)
target = NoisyDQN(n_actions)
target.set_weights(online.get_weights())

optimizer = optimizers.Adam(LR)
mse = tf.keras.losses.MeanSquaredError()

reward_history = []

for episode in range(NUM_EPISODES):

    state, _ = env.reset()
    ep_reward = 0

    for step in range(200):

        # ----------------------------
        # Noisy exploration (no epsilon)
        # ----------------------------
        action = online.act(state)

        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        ep_reward += reward

        buffer.store(state, next_state, action, reward, float(done))
        state = next_state

        # ----------------------------
        # TRAINING
        # ----------------------------
        if buffer.ptr > 1000 or buffer.full:

            states, next_states, actions, rewards, dones = buffer.sample(BATCH)

            next_actions = tf.argmax(online(next_states), axis=1, output_type=tf.int32)
            idx = tf.stack([tf.range(BATCH, dtype=tf.int32), next_actions], axis=1)

            target_Q_all = target(next_states)
            next_Q = tf.gather_nd(target_Q_all, idx)

            y = rewards + GAMMA * (1 - dones) * next_Q

            with tf.GradientTape() as tape:
                q_pred = online(states)
                a_idx = tf.stack(
                    [tf.range(BATCH, dtype=tf.int32), actions], axis=1
                )
                pred = tf.gather_nd(q_pred, a_idx)
                loss = mse(pred, y)

            grads = tape.gradient(loss, online.trainable_variables)
            grads = [tf.clip_by_norm(g, 5.0) for g in grads]
            optimizer.apply_gradients(zip(grads, online.trainable_variables))

            # Soft update
            target_weights = target.get_weights()
            online_weights = online.get_weights()
            tau = 0.005
            new_weights = [
                tw * (1 - tau) + ow * tau
                for tw, ow in zip(target_weights, online_weights)
            ]
            target.set_weights(new_weights)

        if done:
            break

    reward_history.append(ep_reward)

    # ----------------------------
    # LIVE PLOT
    # ----------------------------
    if episode % 5 == 0:
        clear_output(wait=True)
        plt.figure(figsize=(10,4))

        # Reward
        plt.plot(
            reward_history,
            label="Reward",
            color="blue",
            alpha=0.3,  
            linewidth=1
        )

        # Moving average
        if len(reward_history) > 20:
            ma = np.convolve(reward_history, np.ones(20)/10, mode='valid')
            plt.plot(
                range(19, len(reward_history)),
                ma,
                label="Moving Avg (20 eps)",
                color="orange",
                linewidth=2.5
            )

        plt.title("Noisy DQN — Training Progress")
        plt.xlabel("Episode")
        plt.ylabel("Reward")
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.show()
